| Columna/Variable       | Descripción                                                     | Tipo de variable  | Importancia inicial  | Notas |
|------------------------|-----------------------------------------------------------------|-------------------|----------------------|-------|
| Fecha                  | Fecha de generación                                             | Fecha             | 0                    |       |
| Hidraulica             | Generación diaria de energía hidraulica en GWh                  | Numérica continua | 0                    |       |
| Turbinacion bombeo     | Generación diaria de energía por turbinación por bombeo en GWh  | Numérica continua | 0                    |       |
| Nuclear                | Generación diaria de energía nuclear en GWh                     | Numérica continua | 0                    |       |
| Carbon                 | Generación diaria de energía por carbón en GWh                  | Numérica continua | 0                    |       |
| Motores diesel         | Generación diaria de energía por motores diesel en GWh          | Numérica continua | 0                    |       |

In [ ]:
lista_columnas = ["energia_eolica","energia_hidraulica","energia_solar","energia_otras_renovables"]
lista_anyos = df_energia.index.year.unique().to_list()

if len(lista_anyos) % 2 == 0:
    filas = len(lista_anyos) // 2
    
else:
    filas = len(lista_anyos) // 2 + 1

fig, axs = plt.subplots(filas, 2, figsize=(18, filas * 5))

axs = axs.flatten()

#Definimos los colores
color_eolica =  "green"
color_hidraulica = "blue"
color_solar = "orange"
color_otras = "yellow"

for indice, anyo in enumerate(lista_anyos):
    
    df_piechart = df_energia.loc[df_energia.index.year == anyo, lista_columnas]
    
    #Suma de los valores de las tres columnas
    suma_total = df_piechart.sum().sum()

    #Calcular porcentaje de cada columna
    porcentajes = df_piechart.sum() / suma_total * 100
    
    #Crear el pie chart
    axs[indice].pie(porcentajes, autopct='%1.1f%%', startangle=90, counterclock=False, colors = [color_eolica,color_hidraulica,color_solar,color_otras])
    
    axs[indice].legend(porcentajes.index, title="Tipo de energía", loc="upper right", bbox_to_anchor=(1.6, 1))

    #Titulo
    axs[indice].set_title(f"Año {anyo}")

#Quitamos los ejes de la subfigura que no tiene gráfica asignada (si la hay)
for j in range(indice + 1, len(axs)):
    axs[j].axis("off")

plt.suptitle("Distribución de la generación de energía renovable")
plt.show()

In [ ]:
#Definimos una funcion para obtener la principales medidas de tendencia central
def medidas_tendencia_central(df,lista_columnas):
    df_medidas = df.describe()[lista_columnas]
    df_medidas = df_medidas.T
    df_medidas = df_medidas.drop(columns=["count"])
    
    #Incluimos el coeficiente de variacion
    df_medidas["coeficiente_variacion"] = 0.0
    for variable in df_medidas.index:
        cv = round((df_medidas.loc[variable, "mean"]/df_medidas.loc[variable, "std"])*100,2)
        df_medidas.loc[variable, "coeficiente_variacion"] = cv
    
    return df_medidas

In [ ]:
df_piechart = df_energia[["Demanda","generacion_total"]]

# Suma de los valores de las tres columnas
suma_total = df_piechart.sum().sum()

# Calcular porcentaje de cada columna
porcentajes = df_piechart.sum() / suma_total * 100

# Crear el pie chart
plt.pie(porcentajes, labels=porcentajes.index, autopct='%1.1f%%', startangle=90)
plt.title('Deistribución de la generación de energía')
plt.show()

Creamos un DataFrame con la suma de la generación de energía por trimestre y año

In [ ]:
#Hacemos una copia del DataFrame original
df_energia_trim = df_energia.copy()

#Creamos nuevas columnas auxiliares con el año y el trimestre
df_energia_trim["año"] = df_energia_trim.index.year
df_energia_trim["trim"] = df_energia_trim.index.quarter

#Creamos la columna con la concatenacion de trimestre y año
df_energia_trim["año_trim"] = df_energia_trim["trim"].astype(str) + "/" +  df_energia_trim["año"].astype(str)

#Eliminamos las columnas auxiliares creadas inicialmente
df_energia_trim.drop(columns=["año","trim"],inplace=True)

#Agrupamos por esas fechas
df_energia_trim = df_energia_trim.groupby("año_trim").sum()